# <h1 style="text-align: center;"> MHW computing notebook </h1>

This Python notebook computes MHW metrics (per events/per year), and saves them as netCDF files.

## Setup

In [1]:
# Enables modules autoreload (important during development)
%load_ext autoreload
%autoreload 2

# Basic imports
import os

# Define basic paths that may be useful
root_path = os.path.join(os.path.abspath(os.getcwd()), '..')
print(root_path)

/LOCALDATA/ARTHURs_FILES/Code/..


In [2]:
# Basic options
clim_period = (1987, 2021)
detrended = False

## Compute REP MHWs

In [3]:
from pyscripts.load_save_dataset import load_rep

# Load the REP dataset
ds_rep = load_rep(region_selector="balears")
ds_rep

Loaded REP dataset.


<xarray.Dataset> Size: 508MB
Dimensions:  (time: 15340, lat: 69, lon: 120)
Coordinates:
  * lat      (lat) float32 276B 37.65 37.7 37.75 37.8 ... 40.96 41.01 41.06
  * lon      (lon) float32 480B -0.8591 -0.8091 -0.7591 ... 4.996 5.046 5.096
  * time     (time) datetime64[ns] 123kB 1982-01-01 1982-01-02 ... 2023-12-31
Data variables:
    T        (time, lat, lon) float32 508MB dask.array<chunksize=(31, 69, 120), meta=np.ndarray>
Attributes:
    institution:               GOS (Global Ocean Satellite monitoring and mar...
    comment:                   WARNING: some applications are unable to prope...
    history:                   GOS-CMEMS processor V4: new version
    Conventions:               CF-1.4 
    title:                     Mediterranean Sea SST Analysis L4, Reprocessed...
    source:                    [1982-present] ESA CCI SST v.3.0 L3C product (...
    references:                A. Pisano, B. Buongiorno Nardelli, C. Tronconi...
    copernicusmarine_version:  2.2.2

### Whole region

Compute annual metrics for every grip point of REP.

In [ ]:
from pyscripts.mhw_computer import compute_mhw_yearly
from pyscripts.load_save_dataset import save_mhws_dataset

# Lazy compute annual metrics
ds_mhws = compute_mhw_yearly(
    ds_rep,
    using_dataset = "rep",
    clim_period = clim_period,
    detrend = detrended
)

# Perform the computation while saving the annual metrics dataset to .nc file
ds_mhws = save_mhws_dataset(
    ds_mhws,
    ds_type = "yearly",
    dataset_used = "rep",
    region = "balears",
    clim_period = clim_period,
    
    progress_bar = True
)

ds_mhws

Stacking dimensions:  ['lat', 'lon']
Saving MHWs dataset to /LOCALDATA/ARTHURs_FILES/Code/../Data/mhws/yearly/rep_mhws_balears_1987_2021.nc
[##################################      ] | 86% Completed | 57.52 sms

### Spatially averaged REP MHWs

Compute event-wise metrics for region-averaged REP SST.

In [4]:
from pyscripts.mhw_computer import compute_mhw_all_events
from pyscripts.load_save_dataset import save_mhws_dataset

# Average the REP over the whole region
ds_rep_mean = ds_rep.mean(dim=['lon', 'lat'])

# Compute event-wise metrics
ds_mhws = compute_mhw_all_events(
    ds_rep_mean,
    clim_period = clim_period,
    using_dataset = 'rep',
    detrend = detrended
)

# Save the event-wise metrics dataset to .nc file
ds_mhws = save_mhws_dataset(
    ds_mhws,
    ds_type = "all_events",
    dataset_used = "rep_mean",
    detrended = detrended,
    region = "balears",
    clim_period = clim_period,
    
    progress_bar = True
)

ds_mhws

Computing MHWs dataset
[########################################] | 100% Completed | 3.65 sms
Saving MHWs dataset to /LOCALDATA/ARTHURs_FILES/Code/../Data/mhws/all_events/rep_mean_mhws_balears_1987_2021.nc
Created dir /LOCALDATA/ARTHURs_FILES/Code/../Data/mhws/all_events.
 -> Saved!


<xarray.Dataset> Size: 631kB
Dimensions:                         (event_number: 79, time: 15340)
Coordinates:
  * event_number                    (event_number) int64 632B 0 1 2 ... 76 77 78
  * time                            (time) datetime64[ns] 123kB 1982-01-01 .....
Data variables: (12/30)
    time_start                      (event_number) float64 632B 7.241e+05 ......
    time_end                        (event_number) float64 632B 7.241e+05 ......
    time_peak                       (event_number) float64 632B 7.241e+05 ......
    index_start                     (event_number) float64 632B 557.0 ... nan
    index_end                       (event_number) float64 632B 562.0 ... nan
    index_peak                      (event_number) float64 632B 560.0 ... nan
    ...                              ...
    rate_onset                      (event_number) float64 632B 0.1248 ... nan
    rate_decline                    (event_number) float64 632B 0.3481 ... nan
    clim_thresh                     (time) float64 123kB 16.09 16.03 ... 16.15
    clim_seas                       (time) float64 123kB 15.36 15.31 ... 15.41
    clim_missing                    (time) float64 123kB 0.0 0.0 0.0 ... 0.0 0.0
    clim_sst                        (time) float64 123kB 15.29 15.39 ... 15.9
Attributes:
    climatologyPeriod:  1987-2021
    description:        MHWs statistics computed using the marineHeatWaves mo...
    acknowledgment:     Generated using E.U. Copernicus Marine Service Inform...

/LOCALDATA/ARTHURs_FILES/Code/pyscripts/marineHeatWaves.py:475: RuntimeWarning: invalid value encountered in scalar divide
  mhw['rate_decline'].append((mhw_relSeas[tt_peak] - mhw_relSeas[-1]) / (tt_end-tt_start-tt_peak))


## Compute MEDREA MHWs

In [5]:
from pyscripts.load_save_dataset import load_medrea

# Load the MEDREA dataset
ds_medrea = load_medrea(
    region_selector = "balears",
    depth_selector = [0, 50, 100, 150, 200, 500, 700, 1000, 1500, 2000],
)

ds_medrea

Loaded MEDREA dataset.


<xarray.Dataset> Size: 6GB
Dimensions:  (depth: 10, lat: 84, lon: 144, time: 13149)
Coordinates:
  * depth    (depth) float32 40B 1.018 51.38 97.93 ... 1.487e+03 2.001e+03
  * lat      (lat) float32 336B 37.6 37.65 37.69 37.73 ... 40.98 41.02 41.06
  * lon      (lon) float32 576B -0.875 -0.8333 -0.7917 -0.75 ... 5.0 5.042 5.083
  * time     (time) datetime64[ns] 105kB 1987-01-01 1987-01-02 ... 2022-12-31
Data variables:
    T        (time, depth, lat, lon) float32 6GB dask.array<chunksize=(29, 10, 79, 136), meta=np.ndarray>
Attributes:
    contact:                   servicedesk.cmems@mercator-ocean.eu
    comment:                   Please check in CMEMS catalogue the INFO secti...
    institution:               Centro Euro-Mediterraneo sui Cambiamenti Clima...
    Conventions:               CF-1.0
    title:                     Potential Temperature (3D) - Daily Mean
    source:                    MFS E3R1
    references:                Please check in CMEMS catalogue the INFO secti...
    copernicusmarine_version:  2.2.2

### Whole region

Compute annual metrics for every grip point of MEDREA.

In [ ]:
from pyscripts.mhw_computer import compute_mhw_yearly
from pyscripts.load_save_dataset import save_mhws_dataset

# Lazy compute annual metrics
ds_mhws = compute_mhw_yearly(
    ds_medrea,
    using_dataset = "medrea",
    clim_period = clim_period,
    detrend = detrended,
)

# Perform the computation while saving the annual metrics dataset to .nc file
save_mhws_dataset(
    ds_mhws,
    ds_type = "yearly",
    dataset_used = "medrea_detrended" if detrended else "medrea",
    clim_period = clim_period,
    region = "balears",

    progress_bar = True
)

ds_mhws

Stacking dimensions:  ['depth', 'lat', 'lon']
Saving MHWs dataset to /LOCALDATA/ARTHURs_FILES/Code/../Data/mhws/yearly/medrea_mhws_balears_1987_2021.nc
Created dir /LOCALDATA/ARTHURs_FILES/Code/../Data/mhws/yearly.
[                                        ] | 2% Completed | 49.88 sms

### Spatially averaged MEDREA MHWs

Compute event-wise metrics for region-averaged MEDREA temperature for every depth levels.

In [6]:
from pyscripts.mhw_computer import compute_mhw_all_events
from pyscripts.load_save_dataset import save_mhws_dataset

# Average the MEDREA over the whole region
ds_medrea_mean = ds_medrea.mean(dim=['lon', 'lat'])

# Compute event-wise metrics
ds_mhws = compute_mhw_all_events(
    ds_medrea_mean,
    clim_period = clim_period,
    using_dataset = 'medrea',
    detrend = detrended
)

# Save the event-wise metrics dataset to .nc file
ds_mhws = save_mhws_dataset(
    ds_mhws,
    ds_type = "all_events",
    dataset_used = "medrea_mean",
    detrended = detrended,
    region = "balears",
    clim_period = clim_period,
    
    progress_bar = True
)

ds_mhws

Computing MHWs dataset
[########################################] | 100% Completed | 32m 5ss
Saving MHWs dataset to /LOCALDATA/ARTHURs_FILES/Code/../Data/mhws/all_events/medrea_mean_mhws_balears_1987_2021.nc
 -> Saved!


<xarray.Dataset> Size: 4MB
Dimensions:                         (depth: 10, event_number: 67, time: 13149)
Coordinates:
  * depth                           (depth) float32 40B 1.018 ... 2.001e+03
  * event_number                    (event_number) int64 536B 0 1 2 ... 64 65 66
  * time                            (time) datetime64[ns] 105kB 1987-01-01 .....
Data variables: (12/30)
    time_start                      (depth, event_number) float64 5kB 7.256e+...
    time_end                        (depth, event_number) float64 5kB 7.256e+...
    time_peak                       (depth, event_number) float64 5kB 7.256e+...
    index_start                     (depth, event_number) float64 5kB 262.0 ....
    index_end                       (depth, event_number) float64 5kB 277.0 ....
    index_peak                      (depth, event_number) float64 5kB 269.0 ....
    ...                              ...
    rate_onset                      (depth, event_number) float64 5kB 0.06575...
    rate_decline                    (depth, event_number) float64 5kB 0.05134...
    clim_thresh                     (depth, time) float64 1MB 16.04 ... 12.92
    clim_seas                       (depth, time) float64 1MB 15.36 ... 12.84
    clim_missing                    (depth, time) float64 1MB 0.0 0.0 ... 0.0
    clim_sst                        (depth, time) float64 1MB 15.31 ... 12.93
Attributes:
    climatologyPeriod:  1987-2021
    description:        MHWs statistics computed using the marineHeatWaves mo...
    acknowledgment:     Generated using E.U. Copernicus Marine Service Inform...